In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import beamsim
from concurrent.futures import ThreadPoolExecutor, as_completed
from threading import current_thread
import os
import time

# Set seaborn style for beautiful plots
sns.set_style("whitegrid")
sns.set_palette("husl")
plt.rcParams["figure.figsize"] = (12, 8)
plt.rcParams["font.size"] = 11

# Default configuration parameters - single source of truth
DEFAULT_CONFIG = {
    'backend': 'ns3-direct',
    'snark1_pull': True,
    'snark1_half_direct': True,
    'signature_half_direct': 0,
    'shuffle': False,
    'random_seed': 42,
    'group_count': 8,
    'group_validator_count': 1024,
    'group_local_aggregator_count': "10%",  # Default to 10% of group validators
    'global_aggregator_count': 102,
    'mesh_n': 8,
    'non_mesh_n': 4,
    'idontwant': False,
    'signature_time': '20ms',
    'signature_size': 3072,
    'snark_size': 131072,
    'snark1_threshold': 0.9,
    'snark2_threshold': 0.66,
    'aggregation_rate_per_sec': 1000,
    'snark_recursion_aggregation_rate_per_sec': 10,
    'pq_signature_verification_time': '30us',
    'snark_proof_verification_time': '5ms',
    'gml': 'shadow-atlas.bin',
    'max_bitrate': '50Mbps'  # Default EIP-7870 bandwidth limit
}


def generate_yaml_config(topology="gossip", **overrides):
    """
    Generate a YAML configuration with default values and optional overrides.
    
    Args:
        topology: Network topology ("gossip", "grid", etc.)
        **overrides: Dictionary of parameter overrides
        
    Common override parameters:
        - signature_size: Size of signatures in bytes
        - max_bitrate: Bandwidth limit (e.g., "10Mbps" or None for unlimited)
        - mesh_n: Gossip mesh_n parameter
        - non_mesh_n: Gossip non_mesh_n parameter
        - group_validator_count: Number of validators per group
        - group_local_aggregator_count: Number of local aggregators per group
        - group_count: Number of groups
        - global_aggregator_count: Number of global aggregators
        - snark1_threshold: Threshold for SNARK1 generation
        - aggregation_rate_per_sec: Rate of signature aggregation
        - random_seed: Random seed for reproducibility
    """
    
    # Use default configuration and add topology, then apply overrides
    defaults = DEFAULT_CONFIG.copy()
    defaults['topology'] = topology
    
    # Apply overrides
    config = {**defaults, **overrides}
    
    # Build the YAML string
    yaml_content = f"""
# Simulation Backend Configuration
backend: {config['backend']}
                                
snark1_pull: {str(config['snark1_pull']).lower()}
snark1_half_direct: {str(config['snark1_half_direct']).lower()}
signature_half_direct: {str(config['signature_half_direct']).lower()}

# Network Topology Configuration
topology: {config['topology']}

# Whether to shuffle validators from the same group to different routers
shuffle: {str(config['shuffle']).lower()}

# Seed for reproducible simulation results
random_seed: {config['random_seed']}

# Role Assignment Configuration
roles:
  group_count: {config['group_count']}
  group_validator_count: {config['group_validator_count']}
  group_local_aggregator_count: {config['group_local_aggregator_count']}
  global_aggregator_count: {config['global_aggregator_count']}

# Gossipsub Network Configuration
gossip:
  mesh_n: {config['mesh_n']}
  non_mesh_n: {config['non_mesh_n']}
  idontwant: {str(config['idontwant']).lower()}

# Cryptographic Constants
consts:
  signature_time: {config['signature_time']}
  signature_size: {config['signature_size']}
  snark_size: {config['snark_size']}
  snark1_threshold: {config['snark1_threshold']}
  snark2_threshold: {config['snark2_threshold']}
  aggregation_rate_per_sec: {config['aggregation_rate_per_sec']}
  snark_recursion_aggregation_rate_per_sec: {config['snark_recursion_aggregation_rate_per_sec']}
  pq_signature_verification_time: {config['pq_signature_verification_time']}
  snark_proof_verification_time: {config['snark_proof_verification_time']}

# Network Simulation Parameters
network:
  gml: "{config['gml']}"
"""
    
    # Add bandwidth constraint if specified
    if config['max_bitrate'] is not None:
        yaml_content += f"  max_bitrate: {config['max_bitrate']}\n"
    
    return beamsim.yaml(yaml_content.strip())


# Worker function for parallel processing - using ThreadPoolExecutor instead of multiprocessing
# to avoid pickle issues in Jupyter notebooks
def _run_single_simulation_thread(topology, param_value, parameter_name, param_display, 
                                 config_overrides, run_kwargs, display_name, display_unit):
    """
    Worker function to run a single configuration across multiple seeds in parallel context.
    Runs the same configuration 5 times with different random_seed values and returns averaged metrics.
    """
    try:
        # Determine seed values to use
        base_seed = config_overrides.get('random_seed', DEFAULT_CONFIG.get('random_seed', 42))
        seed_values = [base_seed + i for i in range(5)]

        # Determine signature_half_direct for labeling (especially for grid)
        sig_hd_for_display = None
        if topology == 'grid':
            if parameter_name == 'signature_half_direct':
                sig_hd_for_display = param_value
            elif 'signature_half_direct' in config_overrides:
                sig_hd_for_display = config_overrides['signature_half_direct']
            else:
                sig_hd_for_display = DEFAULT_CONFIG.get('signature_half_direct', 0)

        times_ms = []
        dup_counts = []
        dup_avgs = []
        validator_count = None
        snark1_threshold = None

        for seed in seed_values:
            # Create config with this parameter value and seed
            current_config = {**config_overrides, parameter_name: param_value, 'random_seed': seed}
            modified_yaml = generate_yaml_config(topology=topology, **current_config)

            # Run simulation with modified config
            current_run_kwargs = {**run_kwargs, 'c': modified_yaml}
            items = beamsim.run(**current_run_kwargs, t=topology, local_aggregation_only=True)

            # Extract SNARK1 timing information
            snark1_sent = beamsim.get_snark1_sent(items)
            if snark1_sent and len(snark1_sent[0]) > 0:
                snark1_completion_time = snark1_sent[0][-1] if snark1_sent[0] else 0
                times_ms.append(snark1_completion_time)

                # Get simulation info (consistent across seeds)
                _, _, validator_count, snark1_threshold, _ = beamsim.filter_report(items, "info")[0]

                # Extract signature duplicate information (may be absent)
                try:
                    signature_duplicates, avg_duplicates = beamsim.get_signature_duplicates(items)
                except Exception:
                    signature_duplicates = 0
                    avg_duplicates = 0.0
                dup_counts.append(signature_duplicates)
                dup_avgs.append(avg_duplicates)
            # else: skip this seed if no data found

        if len(times_ms) > 0:
            # Determine topology display name - handle idontwant variants for gossip and sig_hd for grid
            base_topology_display = "gossipsub" if topology == "gossip" else beamsim.topology_name.get(topology, topology)
            if topology == 'grid' and sig_hd_for_display is not None:
                topology_display = f"{base_topology_display} (signature_half_direct={sig_hd_for_display})"
            elif " (idontwant=" in param_display and topology == "gossip":
                if "(idontwant=True)" in param_display:
                    topology_display = f"{base_topology_display} (idontwant=True)"
                else:
                    topology_display = f"{base_topology_display} (idontwant=False)"
            else:
                topology_display = base_topology_display

            return {
                'topology': topology,
                'topology_display': topology_display,
                'parameter_name': parameter_name,
                'parameter_value': param_value,
                'parameter_display': param_display,
                'signature_half_direct': sig_hd_for_display,
                # Average across seeds
                'snark1_completion_time': float(np.mean(times_ms)),
                'validator_count': validator_count,
                'snark1_threshold': snark1_threshold,
                'signature_duplicates': float(np.mean(dup_counts)) if dup_counts else 0.0,
                'avg_signature_duplicates': float(np.mean(dup_avgs)) if dup_avgs else 0.0,
                'num_seeds': len(times_ms),
                'success': True
            }
        else:
            return {
                'topology': topology,
                'parameter_name': parameter_name,
                'parameter_value': param_value,
                'parameter_display': param_display,
                'success': False,
                'error': 'No SNARK1 data found across seeds'
            }

    except Exception as e:
        return {
            'topology': topology,
            'parameter_name': parameter_name,
            'parameter_value': param_value,
            'parameter_display': param_display,
            'success': False,
            'error': str(e)
        }


def theoretical_minimum_snark1_time(validator_count, snark1_threshold, signature_size, sig_agg_rate, max_bitrate):
    """
    Calculate the theoretical minimum SNARK1 aggregation time based on parameters.
    
    Args:
        validator_count: Total number of validators
        snark1_threshold: Threshold for SNARK1 generation (0-1)
        signature_size: Size of each signature in bytes
        sig_agg_rate: Rate of signature aggregation per second
        max_bitrate: Maximum bandwidth in bytes per second (None for unlimited)
        
    Returns:
        Theoretical minimum SNARK1 aggregation time in milliseconds
    """
    if max_bitrate is None:
        max_bitrate = float('inf')  # No limit
    
    # Calculate total data size to aggregate
    total_data_size = validator_count * signature_size * snark1_threshold  # in bytes

    # Calculate time to aggregate at the given rate
    agg_time_sec = validator_count * snark1_threshold  / sig_agg_rate  # in seconds

    # Calculate time based on bandwidth limit
    bandwidth_time_sec = total_data_size / max_bitrate  # in seconds
    
    min_time_sec = agg_time_sec + bandwidth_time_sec
    
    return min_time_sec * 1000  # Convert to milliseconds


def run_parameter_analysis(parameter_name, parameter_values, topologies=["gossip", "grid"], 
                          display_name=None, display_unit="", parallel=True, max_workers=None, 
                          **base_config):
    """
    Generic function to analyze the effect of changing a parameter on SNARK1 aggregation time.
    
    Args:
        parameter_name: Name of the parameter to vary (e.g., 'signature_size', 'mesh_n')
        parameter_values: List of values to test for the parameter
        topologies: List of network topologies to test
        display_name: Human-readable name for the parameter (defaults to parameter_name)
        display_unit: Unit to display after the parameter value (e.g., "KB", "Mbps")
        parallel: Whether to run simulations in parallel (default: True)
        max_workers: Maximum number of parallel workers (default: min(8, cpu_count()))
        **base_config: Base configuration parameters and run kwargs
        
    Returns:
        DataFrame with results, with parameter order preserved
    """
    
    if display_name is None:
        display_name = parameter_name.replace('_', ' ').title()
    
    # Separate run kwargs from config overrides
    run_kwargs_keys = {'mpi', 'c', 'b', 'g', 'gv', 'la', 'ga', 'shuffle', 't', 'local_aggregation_only'}
    run_kwargs = {k: v for k, v in base_config.items() if k in run_kwargs_keys}
    config_overrides = {k: v for k, v in base_config.items() if k not in run_kwargs_keys}
    
    # Determine the number of workers (use ThreadPoolExecutor which works better in Jupyter)
    if max_workers is None:
        max_workers = min(8, len(parameter_values) * len(topologies))  # Conservative default
    
    print(f"Testing {display_name}: {parameter_values}")
    print(f"Topologies: {topologies}")
    
    if parallel:
        print(f"Running simulations in parallel with {max_workers} workers (ThreadPoolExecutor)...\n")
    else:
        print("Running simulations sequentially...\n")
    
    # Prepare all simulation parameters
    simulation_params = []
    
    # Check if idontwant flag is set in config_overrides
    idontwant_flag = config_overrides.get('idontwant', None)
    
    for topology in topologies:
        for param_value in parameter_values:
            param_display = f"{param_value} {display_unit}".strip() if param_value is not None else "Default/Unlimited"
            
            # Handle idontwant flag specially for gossip topology
            if topology == "gossip" and idontwant_flag is True:
                # Run gossip with both idontwant=True and idontwant=False
                for idontwant_value in [True, False]:
                    # Create separate config overrides for each idontwant value
                    gossip_config = config_overrides.copy()
                    gossip_config['idontwant'] = idontwant_value
                    
                    # Create display name that includes idontwant status
                    idontwant_suffix = " (idontwant=True)" if idontwant_value else " (idontwant=False)"
                    gossip_param_display = param_display + idontwant_suffix
                    
                    simulation_params.append((
                        topology, param_value, parameter_name, gossip_param_display,
                        gossip_config, run_kwargs, display_name, display_unit
                    ))
            # For grid topology, execute in two modes for signature_half_direct (0 and 4) unless we're varying it directly
            elif topology == "grid" and parameter_name != 'signature_half_direct':
                for sig_hd in [0, 4]:
                    grid_config = config_overrides.copy()
                    grid_config['signature_half_direct'] = sig_hd
                    simulation_params.append((
                        topology, param_value, parameter_name, param_display,
                        grid_config, run_kwargs, display_name, display_unit
                    ))
            else:
                # For grid (when parameter_name is signature_half_direct) or other topologies, run once with original config
                simulation_params.append((
                    topology, param_value, parameter_name, param_display,
                    config_overrides, run_kwargs, display_name, display_unit
                ))
    
    results = []
    
    if parallel and len(simulation_params) > 1:
        # Run simulations in parallel using ThreadPoolExecutor
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            # Submit all tasks
            future_to_params = {}
            for params in simulation_params:
                topology, param_value, parameter_name, param_display, config_overrides, run_kwargs, display_name, display_unit = params
                future = executor.submit(_run_single_simulation_thread, 
                                       topology, param_value, parameter_name, param_display,
                                       config_overrides, run_kwargs, display_name, display_unit)
                future_to_params[future] = params
            
            # Process completed tasks
            total_sims = len(simulation_params)
            completed = 0
            
            for future in as_completed(future_to_params):
                completed += 1
                result = future.result()
                
                if result['success']:
                    results.append(result)
                    label = result.get('topology_display') or beamsim.topology_name.get(result['topology'], result['topology'])
                    seeds_info = result.get('num_seeds', 1)
                    print(f"  [{completed}/{total_sims}] {label} - {result['parameter_display']}: "
                          f"{result['snark1_completion_time']:.1f} ms (avg over {seeds_info} seeds)")
                else:
                    print(f"  [{completed}/{total_sims}] ERROR - {result['topology']} - {result['parameter_display']}: "
                          f"{result['error']}")
    else:
        # Run simulations sequentially (original behavior)
        for i, params in enumerate(simulation_params):
            topology, param_value, parameter_name, param_display, config_overrides, run_kwargs, display_name, display_unit = params
            display_topology = "gossipsub" if topology == "gossip" else beamsim.topology_name.get(topology, topology)
            print(f"  [{i+1}/{len(simulation_params)}] Testing {display_topology} - {param_display}")
            
            result = _run_single_simulation_thread(
                topology, param_value, parameter_name, param_display,
                config_overrides, run_kwargs, display_name, display_unit
            )
            
            if result['success']:
                results.append(result)
                seeds_info = result.get('num_seeds', 1)
                print(f"    → {result['snark1_completion_time']:.1f} ms (avg over {seeds_info} seeds)")                
            else:
                    print(f"    → ERROR: {result['error']}")
    
    # Convert to DataFrame and preserve parameter order
    df = pd.DataFrame(results)
    if not df.empty:
        # Add an order column to preserve the original parameter order
        param_to_order = {param: i for i, param in enumerate(parameter_values)}
        df['parameter_order'] = df['parameter_value'].map(param_to_order)
        # Sort by topology and parameter order to ensure consistent ordering
        df = df.sort_values(['topology', 'parameter_order']).reset_index(drop=True)
    
    return df


def plot_parameter_analysis(results_df, parameter_name, display_name=None, display_unit="", 
                          title_suffix="", figure_size=(14, 12), include_theoretical=True,
                          **base_config):
    """
    Plot the results of a parameter analysis.
    
    Args:
        results_df: DataFrame from run_parameter_analysis
        parameter_name: Name of the parameter that was varied
        display_name: Human-readable name for the parameter
        display_unit: Unit for the parameter
        title_suffix: Additional text for the plot title
        figure_size: Size of the plot figure
        include_theoretical: Whether to include theoretical minimum line
        **base_config: Base configuration parameters for theoretical calculations
    """
    
    if display_name is None:
        display_name = parameter_name.replace('_', ' ').title()
    
    # Create subplots - one for SNARK1 time, one for signature duplicates
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=figure_size)
    
    # Plot lines for each topology display (handles idontwant variations)
    topology_displays = results_df['topology_display'].unique()
    colors = sns.color_palette("husl", len(topology_displays))
    
    # Get parameter order from the dataframe (preserves original input order)
    # Remove idontwant suffixes for parameter ordering
    results_df_clean = results_df.copy()
    results_df_clean['parameter_display_clean'] = results_df_clean['parameter_display'].str.replace(r' \(idontwant=(?:True|False)\)', '', regex=True)
    
    if 'parameter_order' in results_df_clean.columns:
        # Use the preserved order
        ordered_params = results_df_clean.sort_values('parameter_order')['parameter_display_clean'].unique()
    else:
        # Fallback to original behavior
        ordered_params = results_df_clean['parameter_display_clean'].unique()
    
    # Plot 1: SNARK1 Aggregation Time
    plt.sca(ax1)  # Set current axis
    for i, topology_display in enumerate(topology_displays):
        topo_data = results_df[results_df['topology_display'] == topology_display].copy()
        
        # Sort by parameter order to maintain input sequence
        if 'parameter_order' in topo_data.columns:
            topo_data = topo_data.sort_values('parameter_order')
        else:
            # Fallback: try to sort by parameter value if numeric
            try:
                topo_data = topo_data.sort_values('parameter_value')
            except:
                pass  # Keep original order if sorting fails
        
        x_values = list(range(len(topo_data)))
        y_values = topo_data['snark1_completion_time'].tolist()
        
        ax1.plot(
            x_values,
            y_values,
            marker='o',
            linewidth=3,
            markersize=8,
            color=colors[i],
            label=topology_display,
            alpha=0.8
        )
    
    # Add theoretical minimum line if requested
    if include_theoretical and len(results_df) > 0:
        # Get default parameters for theoretical calculation
        # Use the global DEFAULT_CONFIG as the base for theoretical calculations
        theoretical_params = {**DEFAULT_CONFIG, **base_config}
        
        # Convert max_bitrate to bytes per second if it's a string
        max_bitrate = theoretical_params['max_bitrate']
        if isinstance(max_bitrate, str) and max_bitrate.endswith('Mbps'):
            max_bitrate_mbps = float(max_bitrate[:-4])
            max_bitrate_bps = max_bitrate_mbps * 1024 * 1024 / 8  # Convert Mbps to bytes/sec
        elif max_bitrate is None:
            max_bitrate_bps = None
        else:
            max_bitrate_bps = max_bitrate
        
        # Calculate total validator count (group_count * group_validator_count)
        total_validator_count = theoretical_params['group_validator_count']
        
        # Calculate theoretical minimum times for each parameter value
        theoretical_times = []
        sample_data = results_df.sort_values('parameter_order' if 'parameter_order' in results_df.columns else 'parameter_value')
        unique_param_values = sample_data['parameter_value'].unique()
        
        for param_value in unique_param_values:
            # Update the parameter being analyzed
            current_params = theoretical_params.copy()
            current_params[parameter_name] = param_value
            
            # Handle special cases for parameter-specific calculations
            if parameter_name == 'max_bitrate':
                if isinstance(param_value, str) and param_value.endswith('Mbps'):
                    max_bitrate_mbps = float(param_value[:-4])
                    current_max_bitrate = max_bitrate_mbps * 1024 * 1024 / 8
                elif param_value is None:
                    current_max_bitrate = None
                else:
                    current_max_bitrate = param_value
            else:
                current_max_bitrate = max_bitrate_bps
            
            # Recalculate total validator count if group parameters changed
            if parameter_name in ['group_count', 'group_validator_count']:
                current_total_validators = current_params['group_validator_count']
            else:
                current_total_validators = total_validator_count
            
            theoretical_time = theoretical_minimum_snark1_time(
                validator_count=current_total_validators,
                snark1_threshold=current_params['snark1_threshold'],
                signature_size=current_params['signature_size'],
                sig_agg_rate=current_params['aggregation_rate_per_sec'],
                max_bitrate=current_max_bitrate
            )
            theoretical_times.append(theoretical_time)
        
        # Plot theoretical minimum line
        x_values = list(range(len(theoretical_times)))
        ax1.plot(
            x_values,
            theoretical_times,
            linestyle='--',
            linewidth=2,
            color='red',
            label='Theoretical Minimum',
            alpha=0.8
        )
    
    # Set axis labels and title for first plot
    ax1.set_xlabel(f'{display_name} {display_unit}'.strip(), fontweight='bold', fontsize=12)
    ax1.set_ylabel('SNARK1 Aggregation Time (ms)', fontweight='bold', fontsize=12)
    
    title = f'Effect of {display_name} on SNARK1 Aggregation Time'
    if title_suffix:
        title += f'\n{title_suffix}'
    title += '\n(Local Aggregation Only Mode – averages over 5 seeds)'
    ax1.set_title(title, fontweight='bold', fontsize=14)
    
    # Set x-axis tick labels using preserved parameter order
    ax1.set_xticks(range(len(ordered_params)))
    ax1.set_xticklabels(ordered_params, rotation=45)
    
    ax1.legend(frameon=True, fancybox=True, shadow=True, fontsize=11)
    ax1.grid(True, alpha=0.3)
    
    # Add performance annotations for first plot
    if len(results_df) > 0:
        min_time = results_df['snark1_completion_time'].min()
        max_time = results_df['snark1_completion_time'].max()
        time_diff = max_time - min_time
        improvement = (time_diff / max_time) * 100 if max_time else 0
        
        ax1.text(0.02, 0.98, 
                f'Max time difference: {time_diff:.1f}ms\n'
                f'Performance improvement: {improvement:.1f}%',
                transform=ax1.transAxes,
                verticalalignment='top',
                bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.8),
                fontsize=10)
    
    # Plot 2: Average Signature Duplicates per Validator
    plt.sca(ax2)  # Set current axis
    if 'avg_signature_duplicates' in results_df.columns:
        for i, topology_display in enumerate(topology_displays):
            topo_data = results_df[results_df['topology_display'] == topology_display].copy()
            
            # Sort by parameter order to maintain input sequence
            if 'parameter_order' in topo_data.columns:
                topo_data = topo_data.sort_values('parameter_order')
            else:
                try:
                    topo_data = topo_data.sort_values('parameter_value')
                except:
                    pass
            
            x_values = list(range(len(topo_data)))
            y_values = topo_data['avg_signature_duplicates'].tolist()
            
            ax2.plot(
                x_values,
                y_values,
                marker='s',  # Square markers to differentiate from time plot
                linewidth=3,
                markersize=8,
                color=colors[i],
                label=topology_display,
                alpha=0.8
            )
        
        # Set axis labels and title for second plot
        ax2.set_xlabel(f'{display_name} {display_unit}'.strip(), fontweight='bold', fontsize=12)
        ax2.set_ylabel('Average Signature Duplicates per Validator', fontweight='bold', fontsize=12)
        ax2.set_title(f'Effect of {display_name} on Average Signature Duplicates per Validator', fontweight='bold', fontsize=14)
        
        # Set x-axis tick labels
        ax2.set_xticks(range(len(ordered_params)))
        ax2.set_xticklabels(ordered_params, rotation=45)
        
        ax2.legend(frameon=True, fancybox=True, shadow=True, fontsize=11)
        ax2.grid(True, alpha=0.3)
        
        # Add duplicate message statistics
        if len(results_df) > 0:
            min_dups = results_df['avg_signature_duplicates'].min()
            max_dups = results_df['avg_signature_duplicates'].max()
            avg_dups = results_df['avg_signature_duplicates'].mean()
            
            ax2.text(0.02, 0.98, 
                    f'Min avg duplicates: {min_dups:.3f}\n'
                    f'Max avg duplicates: {max_dups:.3f}\n'
                    f'Overall avg: {avg_dups:.3f}',
                    transform=ax2.transAxes,
                    verticalalignment='top',
                    bbox=dict(boxstyle='round', facecolor='lightcoral', alpha=0.8),
                    fontsize=10)
    else:
        # If no duplicate data is available, show a message
        ax2.text(0.5, 0.5, 'Average signature duplicate data not available',
                transform=ax2.transAxes, ha='center', va='center',
                fontsize=14, style='italic')
        ax2.set_xlim(0, 1)
        ax2.set_ylim(0, 1)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary statistics
    print("\n" + "="*60)
    print(f"{display_name.upper()} EFFECT SUMMARY (averages over 5 seeds)")
    print("="*60)
    
    # Print theoretical minimum information if included
    if include_theoretical and len(results_df) > 0:
        print(f"\nTheoretical Minimum:")
        print(f"  SNARK1 time range: {min(theoretical_times):.1f} - {max(theoretical_times):.1f} ms")
        print(f"  Time difference: {max(theoretical_times) - min(theoretical_times):.1f} ms")
        
        # Show theoretical minimum for each parameter value
        sample_data = results_df_clean.sort_values('parameter_order' if 'parameter_order' in results_df_clean.columns else 'parameter_value')
        unique_param_displays = sample_data['parameter_display_clean'].unique()
        for i, param_display in enumerate(unique_param_displays):
            print(f"    {param_display:>12}: {theoretical_times[i]:.1f} ms")
    
    for topology_display in topology_displays:
        topo_data = results_df[results_df['topology_display'] == topology_display]
        if len(topo_data) > 0:
            print(f"\n{topology_display}:")
            print(f"  SNARK1 time range: {topo_data['snark1_completion_time'].min():.1f} - {topo_data['snark1_completion_time'].max():.1f} ms")
            print(f"  Time difference: {topo_data['snark1_completion_time'].max() - topo_data['snark1_completion_time'].min():.1f} ms")
            
            # Add signature duplicate statistics if available
            if 'avg_signature_duplicates' in topo_data.columns:
                print(f"  Avg duplicates per validator range: {topo_data['avg_signature_duplicates'].min():.3f} - {topo_data['avg_signature_duplicates'].max():.3f}")
                print(f"  Overall avg duplicates per validator: {topo_data['avg_signature_duplicates'].mean():.3f}")
                if 'signature_duplicates' in topo_data.columns:
                    print(f"  Total signature duplicates range: {topo_data['signature_duplicates'].min():.0f} - {topo_data['signature_duplicates'].max():.0f} messages")
            
            # Show performance for each parameter value
            for _, row in topo_data.iterrows():
                dup_info = ""
                if 'avg_signature_duplicates' in row:
                    dup_info = f" (avg dups: {row['avg_signature_duplicates']:.3f})"
                # Clean up parameter display for output (remove idontwant suffix)
                clean_param_display = row['parameter_display'].replace(' (idontwant=True)', '').replace(' (idontwant=False)', '')
                print(f"    {clean_param_display:>12}: {row['snark1_completion_time']:.1f} ms{dup_info}")


def analyze_parameter_effect(parameter_name, parameter_values, topologies=["gossip", "grid"], 
                           display_name=None, display_unit="", title_suffix="", 
                           parallel=True, max_workers=None, **base_config):
    """
    Complete analysis workflow: run simulations and plot results.
    
    Args:
        parameter_name: Name of the parameter to vary
        parameter_values: List of values to test
        topologies: List of topologies to test
        display_name: Human-readable parameter name
        display_unit: Unit for the parameter
        title_suffix: Additional text for plot title
        parallel: Whether to run simulations in parallel (default: True)
        max_workers: Maximum number of parallel workers (default: cpu_count())
        **base_config: Base configuration and run parameters
        
    Returns:
        DataFrame with results
    """
    
    # Run the analysis
    results_df = run_parameter_analysis(
        parameter_name, parameter_values, topologies, 
        display_name, display_unit, parallel=parallel, 
        max_workers=max_workers, **base_config
    )
    
    # Plot the results
    plot_parameter_analysis(
        results_df, parameter_name, display_name, 
        display_unit, title_suffix, **base_config
    )
    
    return results_df

In [ ]:
def analyze_bandwidth_effect(bandwidth_limits, signature_size=3072, topologies=["gossip", "grid"], 
                            parallel=True, max_workers=None, **base_config):
    """
    Analyze how max incoming bandwidth affects SNARK1 aggregation time using the refactored framework.
    Each configuration is run across 5 seeds and averaged (handled by core functions).
    
    Args:
        bandwidth_limits: List of bandwidth limits to test (in Mbps, None for unlimited)
        signature_size: Fixed signature size to use (in bytes)
        topologies: List of network topologies to test
        parallel: Whether to run simulations in parallel (default: True)
        max_workers: Maximum number of parallel workers (default: cpu_count())
        **base_config: Base configuration parameters
        
    Returns:
        DataFrame with results
    """
    # Convert bandwidth values to the format expected by the YAML config
    bandwidth_values = []
    for bw in bandwidth_limits:
        if bw is None:
            bandwidth_values.append(None)
        else:
            bandwidth_values.append(f"{bw}Mbps")
    
    return analyze_parameter_effect(
        parameter_name='max_bitrate',
        parameter_values=bandwidth_values,
        topologies=topologies,
        display_name='Max Incoming Bandwidth',
        display_unit='',
        title_suffix=f'Signature Size: {signature_size/1024:.1f} KB',
        parallel=parallel,
        max_workers=max_workers,
        signature_size=signature_size,  # Fixed signature size
        **base_config
    )

In [ ]:
def analyze_signature_size_effect(signature_sizes, topologies=["gossip", "grid"], 
                                 parallel=True, max_workers=None, **base_config):
    """
    Analyze how signature size affects SNARK1 aggregation time and signature duplicates.
    Each configuration is run across 5 different seeds and averaged (handled by core functions).
    
    Args:
        signature_sizes: List of signature sizes to test (in bytes)
        topologies: List of network topologies to test
        parallel: Whether to run simulations in parallel (default: True)
        max_workers: Maximum number of parallel workers (default: cpu_count())
        **base_config: Base configuration parameters
        
    Returns:
        DataFrame with results
    """
    return analyze_parameter_effect(
        parameter_name='signature_size',
        parameter_values=signature_sizes,
        topologies=topologies,
        display_name='Signature Size',
        display_unit='bytes',
        title_suffix='Network Performance and Efficiency Analysis',
        parallel=parallel,
        max_workers=max_workers,
        **base_config,
    )


def analyze_mesh_n_effect(mesh_n_values, topologies=["gossip"], parallel=True, max_workers=None, **base_config):
    """
    Analyze how mesh_n (gossip mesh size) affects SNARK1 aggregation time and signature duplicates.
    Each configuration is run across 5 different seeds and averaged (handled by core functions).
    
    Args:
        mesh_n_values: List of mesh_n values to test
        topologies: List of network topologies to test (typically just "gossip")
        parallel: Whether to run simulations in parallel (default: True)
        max_workers: Maximum number of parallel workers (default: cpu_count())
        **base_config: Base configuration parameters
        
    Returns:
        DataFrame with results
    """
    return analyze_parameter_effect(
        parameter_name='mesh_n',
        parameter_values=mesh_n_values,
        topologies=topologies,
        display_name='Mesh N',
        display_unit='peers',
        title_suffix='Gossip Network Configuration Analysis',
        parallel=parallel,
        max_workers=max_workers,
        **base_config,
    )


def analyze_group_validator_count_effect(validator_counts, topologies=["gossip", "grid"], 
                                       parallel=True, max_workers=None, **base_config):
    """
    Analyze how group_validator_count affects SNARK1 aggregation time and signature duplicates.
    Each configuration is run across 5 different seeds and averaged (handled by core functions).
    
    Args:
        validator_counts: List of validator counts per group to test
        topologies: List of network topologies to test
        parallel: Whether to run simulations in parallel (default: True)
        max_workers: Maximum number of parallel workers (default: cpu_count())
        **base_config: Base configuration parameters
        
    Returns:
        DataFrame with results
    """
    return analyze_parameter_effect(
        parameter_name='group_validator_count',
        parameter_values=validator_counts,
        topologies=topologies,
        display_name='Group Validator Count',
        display_unit='validators',
        title_suffix='Network Scale Analysis',
        parallel=parallel,
        max_workers=max_workers,
        **base_config,
    )


def analyze_local_aggregator_count_effect(aggregator_counts, topologies=["gossip", "grid"], 
                                        parallel=True, max_workers=None, **base_config):
    """
    Analyze how group_local_aggregator_count affects SNARK1 aggregation time and signature duplicates.
    Each configuration is run across 5 different seeds and averaged (handled by core functions).
    
    Args:
        aggregator_counts: List of local aggregator counts per group to test
        topologies: List of network topologies to test
        parallel: Whether to run simulations in parallel (default: True)
        max_workers: Maximum number of parallel workers (default: cpu_count())
        **base_config: Base configuration parameters
        
    Returns:
        DataFrame with results
    """
    return analyze_parameter_effect(
        parameter_name='group_local_aggregator_count',
        parameter_values=aggregator_counts,
        topologies=topologies,
        display_name='Local Aggregator Count',
        display_unit='aggregators',
        title_suffix='Aggregation Capacity Analysis',
        parallel=parallel,
        max_workers=max_workers,
        **base_config,
    )

In [ ]:
topologies = ["gossip", "grid"]

# Signature Size Analysis - Using the refactored approach
# This demonstrates how the new framework simplifies the analysis process

# Test different signature sizes to analyze their effect on SNARK1 aggregation time
signature_sizes = [
    96,      # 96 bytes - very small signature (BLS)
    1024,    # 1 KB - small signature
    2048,    # 2 KB - medium-small signature  
    3072,    # 3 KB - default signature size
    4096,    # 4 KB - medium signature
]

# Base configuration parameters
base_config = {
    'mpi': False,  # Use MPI for faster simulations
    # Add any other base parameters you want to override
}

# Test on both gossip and grid topologies
topologies = ["gossip", "grid"]

print("Analyzing the effect of signature size on SNARK1 aggregation time...")
print("Using the refactored analysis framework for cleaner code.\n")

# Run the signature size effect analysis using the new framework
results_df = analyze_signature_size_effect(signature_sizes, topologies, max_workers=5, idontwant=True, **base_config)

In [ ]:
# Analyze bandwidth limits
bandwidth_limits = [5, 25, 50, 100, 200]
print("\nAnalyzing the effect of max incoming bandwidth on SNARK1 aggregation time...")
bandwidth_results = analyze_bandwidth_effect(
    bandwidth_limits, signature_size=3072, topologies=topologies,
    max_workers=5, idontwant=True, **base_config
)

In [ ]:
# Example analyses for new parameters using the refactored framework
# These demonstrate how easy it is to analyze different parameters

# Example 1: Analyze mesh_n effect (gossip network mesh size)
print("="*60)
print("EXAMPLE 1: Mesh N Analysis")
print("="*60)

mesh_n_values = [4, 6, 8, 10, 12]  # Different mesh sizes
base_config = {'mpi': False, 'signature_size': 3072}

mesh_n_results = analyze_mesh_n_effect(mesh_n_values, topologies=["gossip"], max_workers=5, idontwant=True, **base_config)


# Example 2: Analyze non_mesh_n effect (gossip non-mesh connections)
# print("\n" + "="*60)
# print("EXAMPLE 2: Non-Mesh N Analysis")
# print("="*60)

# non_mesh_n_values = [2, 4, 6, 8, 10]  # Different non-mesh connection counts
# base_config = {'mpi': False, 'signature_size': 3072}

# non_mesh_n_results = analyze_non_mesh_n_effect(non_mesh_n_values, topologies=["gossip"], max_workers=5, **base_config)


# Example 3: Analyze group_validator_count effect (network scale)
print("\n" + "="*60)
print("EXAMPLE 3: Group Validator Count Analysis")
print("="*60)

validator_counts = [128, 256, 512, 768, 1024]  # Different validator counts per group
base_config = {'mpi': False, 'signature_size': 3072}

validator_count_results = analyze_group_validator_count_effect(validator_counts, topologies=["gossip", "grid"], idontwant=True, **base_config)


# Example 4: Analyze local aggregator count effect
print("\n" + "="*60)
print("EXAMPLE 4: Local Aggregator Count Analysis")
print("="*60)

aggregator_counts = [10, 102, 256, 512]  # Different local aggregator counts
base_config = {'mpi': False, 'signature_size': 3072}

aggregator_count_results = analyze_local_aggregator_count_effect(aggregator_counts, topologies=["gossip", "grid"], max_workers=5, idontwant=True, **base_config)

In [ ]:
# Grid search for optimal (mesh_n, signature_half_direct) on gossipsub
from concurrent.futures import ThreadPoolExecutor, as_completed

def _run_gossipsub_single(mesh_n, signature_half_direct, config_overrides, run_kwargs):
    try:
        # Run the same combo across 5 seeds and average
        base_seed = config_overrides.get('random_seed', DEFAULT_CONFIG.get('random_seed', 42))
        seed_values = [base_seed + i for i in range(5)]

        times_ms = []
        dup_counts = []
        dup_avgs = []
        validator_count = None
        snark1_threshold = None

        for seed in seed_values:
            current_config = {**config_overrides, 'mesh_n': mesh_n, 'signature_half_direct': signature_half_direct, 'random_seed': seed}
            modified_yaml = generate_yaml_config(topology='gossip', **current_config)
            current_run_kwargs = {**run_kwargs, 'c': modified_yaml}
            items = beamsim.run(**current_run_kwargs, t='gossip', local_aggregation_only=True)

            snark1_sent = beamsim.get_snark1_sent(items)
            if snark1_sent and len(snark1_sent[0]) > 0:
                snark1_completion_time = snark1_sent[0][-1] if snark1_sent[0] else 0
                times_ms.append(snark1_completion_time)
                _, _, validator_count, snark1_threshold, _ = beamsim.filter_report(items, 'info')[0]
                try:
                    signature_duplicates, avg_duplicates = beamsim.get_signature_duplicates(items)
                except Exception:
                    signature_duplicates, avg_duplicates = 0, 0.0
                dup_counts.append(signature_duplicates)
                dup_avgs.append(avg_duplicates)
            # else skip this seed

        if times_ms:
            return {
                'success': True,
                'mesh_n': mesh_n,
                'signature_half_direct': signature_half_direct,
                'snark1_completion_time': float(np.mean(times_ms)),
                'validator_count': validator_count,
                'snark1_threshold': snark1_threshold,
                'signature_duplicates': float(np.mean(dup_counts)) if dup_counts else 0.0,
                'avg_signature_duplicates': float(np.mean(dup_avgs)) if dup_avgs else 0.0,
                'num_seeds': len(times_ms),
            }
        else:
            return {
                'success': False,
                'mesh_n': mesh_n,
                'signature_half_direct': signature_half_direct,
                'error': 'No SNARK1 data found across seeds',
            }
    except Exception as e:
        return {
            'success': False,
            'mesh_n': mesh_n,
            'signature_half_direct': signature_half_direct,
            'error': str(e),
        }


def run_gossipsub_grid_search(mesh_values, sig_hd_values, parallel=True, max_workers=None, **base_config):
    """
    Run parameter search on gossipsub for mesh_n and signature_half_direct combinations.
    Each combo is executed over 5 seeds and averaged.

    Returns a DataFrame with results and prints progress.
    """
    run_kwargs_keys = {'mpi', 'c', 'b', 'g', 'gv', 'la', 'ga', 'shuffle', 't', 'local_aggregation_only'}
    run_kwargs = {k: v for k, v in base_config.items() if k in run_kwargs_keys}
    config_overrides = {k: v for k, v in base_config.items() if k not in run_kwargs_keys}

    combos = [(m, s) for m in mesh_values for s in sig_hd_values]
    if max_workers is None:
        max_workers = min(8, len(combos))

    print(f"Searching gossipsub over mesh_n={mesh_values} and signature_half_direct={sig_hd_values}")
    print(f"Parallel: {parallel} (workers={max_workers if parallel else 1})\n")

    results = []

    if parallel and len(combos) > 1:
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            futures = {}
            for mesh_n, shd in combos:
                futures[executor.submit(_run_gossipsub_single, mesh_n, shd, config_overrides, run_kwargs)] = (mesh_n, shd)
            total = len(futures)
            done = 0
            for fut in as_completed(futures):
                done += 1
                res = fut.result()
                if res.get('success'):
                    results.append(res)
                    print(f"  [{done}/{total}] gossipsub - mesh_n={res['mesh_n']}, signature_half_direct={res['signature_half_direct']}: {res['snark1_completion_time']:.1f} ms (avg over {res.get('num_seeds', 1)} seeds)")
                else:
                    m, s = futures[fut]
                    print(f"  [{done}/{total}] ERROR - mesh_n={m}, signature_half_direct={s}: {res.get('error')}")
    else:
        for i, (mesh_n, shd) in enumerate(combos, start=1):
            print(f"  [{i}/{len(combos)}] Testing gossipsub - mesh_n={mesh_n}, signature_half_direct={shd}")
            res = _run_gossipsub_single(mesh_n, shd, config_overrides, run_kwargs)
            if res.get('success'):
                results.append(res)
                print(f"    → {res['snark1_completion_time']:.1f} ms (avg over {res.get('num_seeds', 1)} seeds)")
            else:
                print(f"    → ERROR: {res.get('error')}")

    df = pd.DataFrame(results)
    if df.empty:
        print("No successful results produced.")
        return df

    # Identify best configuration (min time, then min duplicates)
    df_sorted = df.sort_values(['snark1_completion_time', 'avg_signature_duplicates', 'mesh_n', 'signature_half_direct']).reset_index(drop=True)
    best = df_sorted.iloc[0].to_dict()
    print("\nBest configuration (gossipsub, averaged over 5 seeds):")
    print(f"  mesh_n={best['mesh_n']}, signature_half_direct={best['signature_half_direct']} → {best['snark1_completion_time']:.1f} ms (avg dups {best['avg_signature_duplicates']:.3f})")

    return df


def plot_gossipsub_grid_search(results_df, value_col='snark1_completion_time', cmap='viridis'):
    """
    Visualize grid search results as heatmaps for time and duplicates (averages over seeds).
    """
    if results_df.empty:
        print("No results to plot.")
        return

    # Ensure proper ordering
    mesh_order = sorted(results_df['mesh_n'].unique())
    shd_order = sorted(results_df['signature_half_direct'].unique())

    pivot_time = results_df.pivot_table(index='mesh_n', columns='signature_half_direct', values='snark1_completion_time', aggfunc='mean').reindex(index=mesh_order, columns=shd_order)
    pivot_dups = results_df.pivot_table(index='mesh_n', columns='signature_half_direct', values='avg_signature_duplicates', aggfunc='mean').reindex(index=mesh_order, columns=shd_order)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

    sns.heatmap(pivot_time, annot=True, fmt='.1f', cmap=cmap, ax=ax1)
    ax1.set_title('Gossipsub: SNARK1 time (ms) by mesh_n × signature_half_direct (avg over 5 seeds)', fontweight='bold')
    ax1.set_xlabel('signature_half_direct')
    ax1.set_ylabel('mesh_n')

    sns.heatmap(pivot_dups, annot=True, fmt='.3f', cmap='magma', ax=ax2)
    ax2.set_title('Gossipsub: Avg signature duplicates by mesh_n × signature_half_direct (avg over 5 seeds)', fontweight='bold')
    ax2.set_xlabel('signature_half_direct')
    ax2.set_ylabel('mesh_n')

    plt.tight_layout()
    plt.show()

In [ ]:
# Execute grid search on gossipsub for the requested values and visualize
mesh_values = [4, 8, 12, 16, 24]
sig_hd_values = [0, 1, 4, 8, 16, 24]

# Base configuration; inherit defaults and optionally override for speed/reproducibility
base_config = {
    'mpi': False,
    'idontwant': False,          # keep behavior consistent with earlier analyses
    'signature_size': DEFAULT_CONFIG['signature_size'],
}

print("Running gossipsub grid search (mesh_n × signature_half_direct)...")
results_grid = run_gossipsub_grid_search(mesh_values, sig_hd_values, parallel=True, max_workers=8, **base_config)

# Visualize
plot_gossipsub_grid_search(results_grid)

# Show top configurations
if not results_grid.empty:
    topk = results_grid.sort_values(['snark1_completion_time','avg_signature_duplicates']).head(5)
    print("\nTop 5 configurations (by SNARK1 time, then duplicates):")
    for _, row in topk.iterrows():
        print(f"  mesh_n={row['mesh_n']:>2}, signature_half_direct={row['signature_half_direct']:>2} → {row['snark1_completion_time']:.1f} ms (avg dups {row['avg_signature_duplicates']:.3f})")